In [1]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import numpy as np
import pandas as pd
import timeit
import tqdm
import sklearn.metrics

torch.manual_seed(2)

In [2]:
%load_ext Cython

In [3]:
%%cython

import numpy as np
np.get_include() # do we need this on colab? 
cimport cython
cimport numpy as np

cdef dict bases={ 'A':<int>0, 'C':<int>1, 'G':<int>2, 'T':<int>3, 'N':<int>4, } 

@cython.boundscheck(False)
def one_hot( str string ):
    cdef np.ndarray[np.float32_t, ndim=2] res = np.zeros( (5,len(string)), dtype=np.float32 )
    cdef int j
    for j in range(len(string)):
        if string[j] in bases: # bases can be 'N' signifying missing: this corresponds to all 0 in the encoding
            res[ bases[ string[j] ], j ]=float(1.0)
    return(res)


In [4]:
x_np = one_hot("CCGCGNGGNGGCAGNNNNN") #ACGTN
x_tensor = torch.tensor(x_np)
print(x_tensor.shape)
torch.sum(x_tensor, 1)

torch.Size([5, 19])


tensor([1., 4., 7., 0., 7.])

In [5]:
data_dir = './data/'
train_data = pd.read_csv(data_dir + 'anchor_datasets/anchor_master_train.csv')
val_data = pd.read_csv(data_dir + 'anchor_datasets/anchor_master_val.csv')
test_data = pd.read_csv(data_dir + 'anchor_datasets/anchor_master_test.csv')

In [6]:
train_data.head()

,Unnamed: 0,chrom,start,end,anchor
0,22021,chr12,54686154,54690154,1.0
1,9664,chr5,32470089,32474089,1.0
2,11985,chr6,42035500,42039500,1.0
3,28325,chr17,62933100,62937100,1.0
4,1887,chr1,153371013,153375013,1.0


In [7]:
import pickle
genome = pickle.load(open(data_dir+"hg38-003.pkl","rb"))

In [8]:
genome["chr6"][42035500:42039500]

'GGGACTACAGGTGCCCACTACCACGCCTGGCTAATTTTTTGTATTTTCAGTAGAGACGGGGTTTCACTGTGTTAGCCAGGATGGTCTCGATCTCCTGACCTCGTGATCCGCCCACCTCAGCCTCCCAAAGTGCTAGGATTACAGGTGTTAGCCACCGTGCCCAGCTAGTTTCTTTTCTTTTCTTTTTTTTTTTTTGAGCCGGAGTCTCACTCTGTCTCCCAGGCCAGAGTACAGTAGCGCGATCTCGGCTCACTGCAACCTCCACCTCCAGGGTTCAAGCAATTCTCCTGCCTCAGCCTCCCAAGTAGCTGGGATTACAGACACCCGCCACAACGCCTGGCTAATTTTTGTATTTTTAGTAGAGACGGGGTTTCACCATTTTGGCCAGGCTGGTTTTGAACTCCTGACTTTGTGATCCCCCCACCTCGGCCTCCCAAAGTGCTGGGATTATAGGTGTGAGCCATCACTCCCGGCCTAAAAATTATTATTATTATTTTGGGTTTTTTTTGAGACAGAGTCTTGCTCTGTCACCCAGGCTGGAGTGCATTGGCCCAATGTTAGCTTGCTGCGCAACCTCTGCCTCTCGGGTTCAAGCGATTCTGCTGCCTCCACCTCCCAAGTAGTTGGGATTATAGGCATGTGCCATCATGCCCAGCTAATTCTTGTATTTTCAGTAGACATGTGGTTTCACCATGTTGGCCAGGCTAGTCCTGAACTCCTGACCTCAAGTGATCCATCTGCCTCGGCCTCCCAAAGTGCTGGGATTACAGGTGTGAGCCACCATGCCTGGCCTAAAAATTATTATTTATATATATATATATATAAAATATATATATGGAGCCTTGCTCTTGTTGCCCAGGCTGGAGTGCATATATACTATATATATATATATATATATATATATTTTTTTTTTTTTTTTTTTTTTTTGACACAGGGTCTCCCTCTGTCACCCAGGCTGGAGTGCAGTGGCACAATCACTACTCACTGCAGCCTCCACCT

In [9]:
len(genome["chr10"])

133797422

In [10]:
# cnt=0
# context_length = 4000
# for i,row in enumerate(test_data.itertuples()):
#     midpoint = int(.5 * (row.start + row.end))
#     seq = genome[row.chrom][midpoint - context_length//2:midpoint + context_length//2]
#     if(len(seq) < 4000):
#         pass
#     else:
#         cnt+=1
# #         seq2 = genome[row.chrom][row.start:row.end]
# #         print(len(seq),len(seq2), row.anchor, len(genome[row.chrom]))
# cnt

In [11]:
class AnchorDataset(torch.utils.data.IterableDataset):

    def __init__(self, anchor_data, genome, rnn_length):
        super().__init__()
        self.rnn_len = rnn_length
        self.anchor_data = anchor_data
        self.genome = genome
        self.context_len = 4000
        
    def __iter__(self): 
        for i,row in enumerate(self.anchor_data.itertuples()):
            lab = row.anchor
            midpoint = int(.5 * (row.start + row.end))
            seq = self.genome[row.chrom][ midpoint - self.context_len//2:midpoint + self.context_len//2]
            if len(seq) < self.context_len:
                continue
            
            oh_cnn_seq = one_hot(seq).transpose() # 4k, 5
            #for rnn
            rnn_start = int(self.context_len / 2 - self.rnn_len / 2)
            rnn_end = int(self.context_len / 2 + self.rnn_len / 2)
            rnn_seq = seq[rnn_start: rnn_end]
            oh_rnn_seq = one_hot(rnn_seq).transpose()
            
            yield(oh_cnn_seq, oh_rnn_seq, np.float32(lab))
            

In [12]:
class Anchor_LSTM_Model(nn.Module):
    def __init__(self, in_dim=5, hid_dim=128, out_dim=1):
        super().__init__()
        self.lstm1 = nn.LSTM(in_dim, hid_dim, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hid_dim * 2, hid_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hid_dim * 2, out_dim)
        
    def forward(self, x):
        batch_size = x.shape[0]
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        return self.fc(x).view(batch_size, -1)

    
class Anchor_CNN_Model(nn.Module):
    def __init__(self, layer1_out=256, layer2_out=512, dropout=0.2):
        super().__init__()
        self.seq_len = 4000
        
        self.layer1 = nn.Sequential(
                        nn.Conv2d(in_channels=1, out_channels=layer1_out, kernel_size=(17,5)),
                        nn.BatchNorm2d(layer1_out),
                        nn.LeakyReLU(0.2),
                        nn.Dropout2d(dropout),
                    )
        
        #dilated conv dilation=1
        self.layer2_1 = nn.Sequential(
                        nn.Conv2d(in_channels=layer1_out, out_channels=layer2_out, kernel_size=(5,1), dilation=1, padding='same'),
                        nn.BatchNorm2d(layer2_out),
                        nn.LeakyReLU(0.2),
                        nn.MaxPool2d(kernel_size=(self.seq_len - 16, 1), stride=(self.seq_len - 16, 1)),
                        nn.Dropout2d(dropout),
                    )
        
        #dilated conv dilation=3
        self.layer2_2 = nn.Sequential(
                        nn.Conv2d(in_channels=layer1_out, out_channels=layer2_out, kernel_size=(5,1), dilation=3, padding='same'),
                        nn.BatchNorm2d(layer2_out),
                        nn.LeakyReLU(0.2),
                        nn.MaxPool2d(kernel_size=(self.seq_len - 16, 1), stride=(self.seq_len - 16, 1)),
                        nn.Dropout2d(dropout),
                    )
        
        #dilated conv dilation=7
        self.layer2_3 = nn.Sequential(
                        nn.Conv2d(in_channels=layer1_out, out_channels=layer2_out, kernel_size=(5,1), dilation=7, padding='same'),
                        nn.BatchNorm2d(layer2_out),
                        nn.LeakyReLU(0.2),
                        nn.MaxPool2d(kernel_size=(self.seq_len - 16, 1), stride=(self.seq_len - 16, 1)),
                        nn.Dropout2d(dropout),
                    )
        
    def forward(self, x):
        x = x.unsqueeze(1) #[*, 1, 4k, 5]
        batch_size = x.shape[0]
        x = self.layer1(x)
        o1 = self.layer2_1(x).view(batch_size, -1)
        o2 = self.layer2_2(x).view(batch_size, -1)
        o3 = self.layer2_3(x).view(batch_size, -1)
        out = torch.cat((o1,o2,o3), -1)
        return out
        
        
class Anchor_CNN_LSTM(nn.Module):
    def __init__(self,
                 use_lstm=True,
                 use_cnn=True):
        super().__init__()
        
        out_dim = 0
        
        if use_lstm == True:
            self.lstm_model = Anchor_LSTM_Model()
            out_dim += 800
            
        if use_cnn == True:
            self.cnn_model = Anchor_CNN_Model()
            out_dim += 1536
        
        
        self.classifier = nn.Sequential(
                            nn.Linear(out_dim, 512),
                            nn.BatchNorm1d(512),
                            nn.LeakyReLU(0.2),
                            nn.Dropout(0.2),
            
                            nn.Linear(512, 256),
                            nn.BatchNorm1d(256),
                            nn.LeakyReLU(0.2),
                            nn.Dropout(0.2),
                            
                            nn.Linear(256, 1),
                            nn.Sigmoid()
                        )

    def forward(self, x_cnn, x_rnn):
        x2 = self.cnn_model(x_cnn)
        x1 = self.lstm_model(x_rnn)

        x = torch.cat((x2, x1), -1)

        out = self.classifier(x)

        return out
        

In [13]:
# batch_size = 64
# train_dataset = AnchorDataset(train_data, genome, 800)
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers = 0) 72302

# val_dataset = AnchorDataset(val_data, genome, 800)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers = 0) 1600

# test_dataset = AnchorDataset(test_data, genome, 800)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers = 0) 5670


In [16]:
def run_one_epoch(train_flag, dataloader, cnn_1d, optimizer, device, dataset_size, epoch):

    torch.set_grad_enabled(train_flag)
    cnn_1d.train() if train_flag else cnn_1d.eval() 

    losses = []
    accuracies = []
    auprcs = []
    with tqdm.tqdm(enumerate(dataloader), total=dataset_size, unit="batch") as tepoch:
        for idx, (x_cnn, x_rnn, y) in tepoch: #tqdm.tqdm(enumerate(dataloader), total=dataset_size): # collection of tuples with iterator
            tepoch.set_description(f"Epoch {epoch}")
            (x_cnn, x_rnn, y) = ( x_cnn.to(device), x_rnn.to(device), y.to(device) ) # transfer data to GPU

            output = cnn_1d(x_cnn, x_rnn) # forward pass
            output = output.squeeze() # remove spurious channel dimension
            loss = F.binary_cross_entropy_with_logits( output, y ) # numerically stable

            if train_flag: 
                loss.backward() # back propagation
                optimizer.step()
                optimizer.zero_grad()

            losses.append(loss.detach().cpu().numpy())
            accuracy = torch.mean( ( (output > .5) == (y > .5) ).float() )
#             print('idx',idx, loss.item(), y, output)
            auprc = sklearn.metrics.average_precision_score(y.detach().cpu().numpy(), 
                                                            output.detach().cpu().numpy())
            auprcs.append(auprc)
            accuracies.append(accuracy.detach().cpu().numpy())  
            tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy.detach().cpu().numpy(), auprc=100. * auprc)
            
    
    return( np.mean(losses), np.mean(accuracies) )

In [19]:
def train_model(cnn_1d, train_data, validation_data, epochs=15, patience=4, verbose = True):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_1d = cnn_1d.to(device)

    batch_size = 64
    train_dataset = AnchorDataset(train_data, genome, 800)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers = 16)
    
    val_dataset = AnchorDataset(val_data, genome, 800)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers = 16)

    optimizer = torch.optim.RMSprop(cnn_1d.parameters())
    
    train_accs = []
    val_accs = []
    patience_counter = patience
    best_val_loss = np.inf
    check_point_filename = 'anchor_model_checkpoint.pt' # to save the best model fit to date
    for epoch in range(epochs):
        start_time = timeit.default_timer()
        
        train_loss, train_acc = run_one_epoch(True, train_dataloader, cnn_1d, optimizer, device, 72302, epoch)
        val_loss, val_acc = run_one_epoch(False, validation_dataloader, cnn_1d, optimizer, device, 1600, epoch)
        
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        if val_loss < best_val_loss: 
            torch.save(cnn_1d.state_dict(), check_point_filename)
            best_val_loss = val_loss
            patience_counter = patience
        else: 
            patience_counter -= 1
            if patience_counter <= 0: 
                cnn_1d.load_state_dict(torch.load(check_point_filename)) # recover the best model so far
                break
        elapsed = float(timeit.default_timer() - start_time)
        
        if verbose == True:
            print("Epoch %i took %.2fs. Train loss: %.4f acc: %.4f. Val loss: %.4f acc: %.4f. Patience left: %i" % 
                  (epoch+1, elapsed, train_loss, train_acc, val_loss, val_acc, patience_counter ))
    
    return cnn_1d, train_accs, val_accs

In [ ]:
my_cnn1d = Anchor_CNN_LSTM()
my_cnn1d, train_accs, val_accs = train_model(my_cnn1d, train_data, val_data)

Epoch 0:   0%|          | 236/72302 [04:13<21:33:21,  1.08s/batch, accuracy=100, auprc=100, loss=0.313]

In [40]:
model = Anchor_CNN_LSTM()

In [42]:
model(dummy3, dummy).shape

torch.Size([10, 1])

In [23]:
m=Anchor_LSTM_Model()

In [39]:
dummy = torch.rand((10, 800, 5))

In [25]:
b1 = m(dummy)

In [26]:
b1.shape

torch.Size([10, 800])

In [27]:
m2=Anchor_CNN_Model()

In [38]:
dummy3=torch.rand((10,4000,5))
# b2=m2(dummy3)

In [29]:
b2.shape

torch.Size([10, 1536])

In [30]:
dummy4=torch.cat((b2,b1), -1)
dummy4.shape

torch.Size([10, 2336])

In [36]:
classifier = nn.Sequential(
                            nn.Linear(5536, 512),
                            nn.BatchNorm1d(512),
                            nn.LeakyReLU(0.2),
                            nn.Dropout(0.2),
            
                            nn.Linear(512, 256),
                            nn.BatchNorm1d(256),
                            nn.LeakyReLU(0.2),
                            nn.Dropout(0.2),
                            
                            nn.Linear(256, 1),
                            nn.Sigmoid()
                        )

In [35]:
classifier(dummy4).shape

NameError: name 'classifier' is not defined

In [95]:
dummy2=torch.rand((10, 1, 4000, 5))
dummy2.shape

torch.Size([10, 1, 4000, 5])

In [103]:
layer1_out=256
layer2_out=512
dropout=0.2
layer1 = nn.Sequential(
                        nn.Conv2d(in_channels=1, out_channels=layer1_out, kernel_size=(17,5)),
                        nn.BatchNorm2d(layer1_out),
                        nn.LeakyReLU(0.2),
                        nn.Dropout2d(dropout),
                    )

In [114]:
print(dummy2.shape)
o1=layer1(dummy2)
o1.shape

torch.Size([10, 1, 4000, 5])


torch.Size([10, 256, 3984, 1])

In [130]:
layer2_1 = nn.Sequential(
                        nn.Conv2d(in_channels=layer1_out, out_channels=layer2_out, kernel_size=(5,1), dilation=1, padding='same'),
                        nn.BatchNorm2d(layer2_out),
                        nn.LeakyReLU(0.2),
                        nn.MaxPool2d(kernel_size=(4000 - 16, 1), stride=(4000 - 16, 1)),
                        nn.Dropout2d(dropout),
                    )

In [124]:
o1.shape

torch.Size([10, 256, 3984, 1])

In [127]:
dummy3=torch.rand((10, 256, 3984, 1))
ll=nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(5,1), dilation=1)

In [131]:
layer2_1(o1).shape

torch.Size([10, 512, 1, 1])